### 1.1 Preprocessing - Reinhard Normalization and WSI Tiling

As a first preprocessing step, all slides were color normalized with respect to a reference image selected by an expert neuropathologist. Color normalization was performed using the method described by [Reinhard et. al](https://ieeexplore.ieee.org/document/946629).

The resulting color normalized whole slide images were tiled using PyVips to generate 1536 x 1536 images patches.

In [1]:
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pyvips as Vips
from tqdm import tqdm

from utils import vips_utils, normalize

In [2]:
TRAIN_WSI_DIR = 'data/Dataset 1a Development_train/'              # WSIs in the training set
#VAL_WSI_DIR = 'data/Dataset 1b Development_validation/'           # WSIs in the validation set
TEST_WSI_DIR = 'data/box_Ab/'

SAVE_DIR = 'data/norm_tiles/'

In [3]:
if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

In [4]:
ref_imagename = 'NA5002_2AB.svs'
#ref_imagename = 'NA3777-02_AB.svs'

In [5]:
wsi_train = os.listdir(TRAIN_WSI_DIR)
#wsi_val = os.listdir(VAL_WSI_DIR)
wsi_test = [pathname.split('/')[-1] for pathname in glob.glob(TEST_WSI_DIR+"*.svs")]

#imagenames = sorted(wsi_val + wsi_train)
imagenames = sorted(wsi_test + [ref_imagename])
#imagenames.remove('NA5005-02_AB.svs')             # this WSI was digitalized at 40x, need resize down to 20x
#imagenames.append('NA5005-02_AB.svs')
print(imagenames)

['NA3777-02_AB.svs', 'NA4077-02_AB.svs', 'NA4092-02_AB.svs', 'NA4107-02_AB.svs', 'NA4160-02_AB.svs', 'NA4195-02_AB.svs', 'NA4256-02_AB.svs', 'NA4299-02_AB.svs', 'NA4391-02_AB.svs', 'NA4450-02_AB.svs', 'NA4463-02_AB.svs', 'NA4471-02_AB.svs', 'NA4553-02_AB.svs', 'NA4626-02_AB.svs', 'NA4672-02_AB.svs', 'NA4675-02_AB.svs', 'NA4691-02_AB.svs', 'NA4695-02_AB.svs', 'NA5002_2AB.svs']


In [6]:
%%time
# Load reference image, fit Reinhard normalizer
ref_image = Vips.Image.new_from_file(TRAIN_WSI_DIR + ref_imagename, level=0)
#ref_image = Vips.Image.new_from_file(TEST_WSI_DIR + ref_imagename, level=0)

normalizer = normalize.Reinhard()
normalizer.fit(ref_image)

CPU times: user 38min 35s, sys: 3min 21s, total: 41min 56s
Wall time: 5min 7s


In [8]:
# Remedy on NA4471-02_AB

stats_dict = {}
for imagename in tqdm(imagenames[:-1]):
    if imagename != "NA4471-02_AB.svs":
        continue
    try:
        vips_img = Vips.Image.new_from_file(TEST_WSI_DIR + imagename, level=0)
        print("Loaded Image: " + TEST_WSI_DIR + imagename)
        #vips_img = Vips.Image.new_from_file(TEST_WSI_DIR + imagename, level=0)
    except:
        #vips_img = Vips.Image.new_from_file(VAL_WSI_DIR + imagename, level=0)
        vips_img = Vips.Image.new_from_file(TRAIN_WSI_DIR + imagename, level=0)
        print("Loaded Image: " + TRAIN_WSI_DIR + imagename)
    out = normalizer.transform(vips_img)
    out.filename = vips_img.filename
    vips_utils.save_and_tile(out, SAVE_DIR)
    stats_dict[imagename] = normalizer.image_stats


  0%|          | 0/18 [00:00<?, ?it/s]

Loaded Image: data/box_Ab/NA4471-02_AB.svs



100%|██████████| 18/18 [08:24<00:00, 28.04s/it]


In [7]:
stats_dict = {}
for imagename in tqdm(imagenames[:-1]):
    try:
        vips_img = Vips.Image.new_from_file(TEST_WSI_DIR + imagename, level=0)
        print("Loaded Image: " + TRAIN_WSI_DIR + imagename)
        #vips_img = Vips.Image.new_from_file(TEST_WSI_DIR + imagename, level=0)
    except:
        #vips_img = Vips.Image.new_from_file(VAL_WSI_DIR + imagename, level=0)
        vips_img = Vips.Image.new_from_file(TRAIN_WSI_DIR + imagename, level=0)
        print("Loaded Image: " + TEST_WSI_DIR + imagename)
    out = normalizer.transform(vips_img)
    out.filename = vips_img.filename
    vips_utils.save_and_tile(out, SAVE_DIR)
    stats_dict[imagename] = normalizer.image_stats

  0%|          | 0/18 [00:00<?, ?it/s]

Loaded Image: data/box/NA3777-02_AB.svs


  6%|▌         | 1/18 [07:04<2:00:09, 424.07s/it]

Loaded Image: data/box/NA4077-02_AB.svs


 11%|█         | 2/18 [16:37<2:05:01, 468.82s/it]

...

 89%|████████▉ | 16/18 [1:56:03<13:16, 398.20s/it]

Loaded Image: data/box/NA4691-02_AB.svs


 94%|█████████▍| 17/18 [2:02:25<06:33, 393.11s/it]

Loaded Image: data/box/NA4695-02_AB.svs


100%|██████████| 18/18 [2:09:02<00:00, 394.54s/it]


In [8]:
# Resize the single 40x image down to 20x
# for imagename in tqdm(imagenames[-1:]):
#     vips_img = Vips.Image.new_from_file(TRAIN_WSI_DIR + imagename, level=0)
#     vips_img = vips_img.resize(0.5)
#     out = normalizer.transform(vips_img)
#     out.filename = vips_img.filename
#     vips_utils.save_and_tile(out, SAVE_DIR)
#     stats_dict[imagename] = normalizer.image_stats

In [9]:
import pandas as pd
stats = pd.DataFrame(stats_dict)

In [10]:
stats = stats.transpose()

In [11]:
stats.columns = 'means', 'stds'

In [12]:
print(stats)

                                                              means  \
NA3777-02_AB.svs  (84.8768307838968, 1.8838342760856104, 6.54897...   
NA4077-02_AB.svs  (91.3131558033885, 0.7904653477813409, 0.51400...   
NA4092-02_AB.svs  (87.88871953616047, 1.1075666993207716, 0.8033...   
NA4107-02_AB.svs  (90.94294739155603, 0.5887239399026774, 0.9401...   
NA4160-02_AB.svs  (88.89899260669029, 0.9974418293237802, 0.3308...   
NA4195-02_AB.svs  (86.47197838382841, 1.6366618861228261, -0.850...   
NA4256-02_AB.svs  (87.56450328065948, 0.8650946384410637, 0.6600...   
NA4299-02_AB.svs  (86.76713213578492, 1.450629252548881, -1.0392...   
NA4391-02_AB.svs  (82.77101907259605, 1.8684124681141927, -2.024...   
NA4450-02_AB.svs  (88.85473144493126, 0.6891529168668056, 0.2871...   
NA4463-02_AB.svs  (87.63791836872299, 1.2089569343004831, -2.268...   
NA4471-02_AB.svs  (91.11351936053985, 0.6167910832749284, 1.1381...   
NA4553-02_AB.svs  (89.44513337546904, 0.8088347968056662, -2.102...   
NA4626